<a href="https://colab.research.google.com/github/yashrajkakkad/ml-spotify/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml-spotify/data_merged.csv')
df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence,year,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,minor,major,acousticness_ar,danceability_ar,duration_ms_ar,energy_ar,instrumentalness_ar,liveness_ar,loudness_ar,speechiness_ar,tempo_ar,valence_ar,popularity_ar,count,key_0_ar,key_1_ar,key_2_ar,key_3_ar,key_4_ar,key_5_ar,key_6_ar,key_7_ar,key_8_ar,key_9_ar,key_10_ar,key_11_ar,minor_ar,major_ar,acousticness_yr,danceability_yr,duration_ms_yr,energy_yr,instrumentalness_yr,liveness_yr,loudness_yr,speechiness_yr,tempo_yr,valence_yr,popularity_yr,mode,key_0_yr,key_1_yr,key_2_yr,key_3_yr,key_4_yr,key_5_yr,key_6_yr
0,0.0131,0.2560,182347,0.895,0,0.000106,0.0821,-4.860,29,0.0707,191.307,0.5550,1978,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.145175,0.523720,221014.800000,0.726840,0.023547,0.174298,-7.711613,0.045116,132.180853,0.701493,36.833333,150.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.341064,0.538997,247988.145057,0.552657,0.103383,0.217524,-11.095111,0.064961,120.209319,0.583424,35.272231,1,0,0,1,0,0,0,0
1,0.9800,0.2770,206972,0.145,0,0.879000,0.1110,-19.898,0,0.0845,75.644,0.4940,1953,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.962233,0.464711,199313.805031,0.217114,0.549370,0.166036,-14.844384,0.071003,102.967642,0.539736,2.974843,159.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.891747,0.440834,216552.259500,0.267418,0.314625,0.221442,-15.414304,0.092591,110.008113,0.432251,3.672500,1,1,0,0,0,0,0,0
2,0.7950,0.6850,314667,0.483,0,0.878000,0.1130,-10.202,1,0.0337,97.694,0.8540,1954,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.866200,0.630400,291778.600000,0.567800,0.895600,0.114200,-10.003400,0.040800,117.398600,0.837800,6.800000,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.867413,0.460371,225951.758500,0.263561,0.291320,0.216008,-15.342991,0.103243,108.561912,0.447291,7.707000,1,0,0,0,1,0,0,0
3,0.6560,0.7880,179747,0.808,0,0.000000,0.1540,-6.590,0,0.0395,113.046,0.9690,1954,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.721667,0.739333,175231.000000,0.745000,0.000001,0.077933,-4.730333,0.138767,127.433000,0.929667,2.000000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.867413,0.460371,225951.758500,0.263561,0.291320,0.216008,-15.342991,0.103243,108.561912,0.447291,7.707000,1,0,0,0,1,0,0,0
4,0.3020,0.0753,498560,0.150,0,0.884000,0.1210,-16.705,0,0.0371,76.558,0.0351,1951,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.817780,0.196522,521790.596429,0.309415,0.665675,0.246221,-15.794201,0.041948,94.257485,0.111619,1.666667,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.900334,0.461423,212773.310500,0.257270,0.303290,0.219540,-15.724956,0.107351,109.569882,0.443625,3.419500,1,1,0,0,0,0,0,0


In [ ]:
# df = df.rename(columns={"mode": "audio_mode"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172230 entries, 0 to 172229
Data columns (total 72 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   acousticness         172230 non-null  float64
 1   danceability         172230 non-null  float64
 2   duration_ms          172230 non-null  int64  
 3   energy               172230 non-null  float64
 4   explicit             172230 non-null  int64  
 5   instrumentalness     172230 non-null  float64
 6   liveness             172230 non-null  float64
 7   loudness             172230 non-null  float64
 8   popularity           172230 non-null  int64  
 9   speechiness          172230 non-null  float64
 10  tempo                172230 non-null  float64
 11  valence              172230 non-null  float64
 12  year                 172230 non-null  int64  
 13  key_0                172230 non-null  int64  
 14  key_1                172230 non-null  int64  
 15  key_2            

In [ ]:
mean_popularity = 50
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df.popularity.value_counts()  

0    143561
1     28669
Name: popularity, dtype: int64

In [ ]:
y = df["popularity"].values
X = df.drop(["popularity"],axis=1)

In [ ]:
X.shape, y.shape

((172230, 71), (172230,))

In [ ]:
%%time
input_scaler = StandardScaler()
X_normalized = input_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.30, random_state=42, stratify=y
)

# create target scaler object
# target_scaler = StandardScaler()
# target_scaler.fit(y_train)
# y_train = target_scaler.transform(y_train)
# y_test = target_scaler.transform(y_test)

CPU times: user 491 ms, sys: 42 ms, total: 533 ms
Wall time: 537 ms


In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report


svm = SVC(kernel='rbf',
        #   C=1000,
        #   gamma=0.001,
          random_state=42,
          cache_size=10000)
svm.fit(X_train[:10000], y_train[:10000])

print("Train accuracy of svm algo:", svm.score(X_train[:10000], y_train[:10000]))
print("Test accuracy of svm algo:", svm.score(X_test, y_test))

cv_scores = cross_val_score(svm, X_train[:20000], y_train[:20000])
print("Cross Validation scores - ", cv_scores)

# y_train_pred = cross_val_predict(svm, X_train[:20000], y_train[:20000])

# confusion_matrix = confusion_matrix(y_train[:20000], y_train_pred)
# print("Confusion matrix\n", confusion_matrix)

# p_score = precision_score(y_train[:20000], y_train_pred)
# rc_score = recall_score(y_train[:20000], y_train_pred)
# f1_score = f1_score(y_train[:20000], y_train_pred)
# p_score, rc_score, f1_score
# print("Recall score = ", rc_score)
# print("Precision score = ", p_score)
# print("F1 score = ", f1_score)

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))

Train accuracy of svm algo: 0.9274
Test accuracy of svm algo: 0.9112427180707968
Cross Validation scores -  [0.917   0.91625 0.91225 0.91325 0.91425]
[[42228   840]
 [ 3746  4855]]
Classification report

              precision    recall  f1-score   support

           0       0.92      0.98      0.95     43068
           1       0.85      0.56      0.68      8601

    accuracy                           0.91     51669
   macro avg       0.89      0.77      0.81     51669
weighted avg       0.91      0.91      0.90     51669

CPU times: user 2min 1s, sys: 191 ms, total: 2min 1s
Wall time: 2min 1s
